In [2]:
from netCDF4 import Dataset, num2date
import numpy as np
import json

In [3]:
# importa o Dataset
dataset = Dataset('modelo_paulo.nc')

In [4]:
# informacoes sobre o Dataset
print(dataset)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    TITLE:  OUTPUT FROM WRF V4.1.2 MODEL
    START_DATE: 2020-10-13_12:00:00
    SIMULATION_START_DATE: 2020-10-13_12:00:00
    WEST-EAST_GRID_DIMENSION: 130
    SOUTH-NORTH_GRID_DIMENSION: 140
    BOTTOM-TOP_GRID_DIMENSION: 45
    DX: 18000.0
    DY: 18000.0
    AERCU_OPT: 0
    AERCU_FCT: 1.0
    IDEAL_CASE: 0
    DIFF_6TH_SLOPEOPT: 0
    AUTO_LEVELS_OPT: 2
    DIFF_6TH_THRESH: 0.1
    DZBOT: 50.0
    DZSTRETCH_S: 1.3
    DZSTRETCH_U: 1.1
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 1
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 0
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 10
    RA_LW_PHYSICS: 5
    RA_SW_PHYSICS: 5
    SF_SFCLAY_PHYSICS: 1
    SF_SURFACE_PHYSICS: 2
    BL_PBL_PHYSICS: 1
    CU_PHYSICS: 3
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 1
    SST_UPDATE: 0
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0

In [72]:
# obtem as dimensions disponiveis
print(dataset.dimensions.keys())

dict_keys(['Time', 'DateStrLen', 'west_east', 'south_north', 'bottom_top', 'bottom_top_stag', 'soil_layers_stag', 'west_east_stag', 'south_north_stag', 'seed_dim_stag'])


In [6]:
# obtem as variaveis disponiveis
print(dataset.variables.keys())

dict_keys(['Times', 'XLAT', 'XLONG', 'LU_INDEX', 'ZNU', 'ZNW', 'ZS', 'DZS', 'VAR_SSO', 'U', 'V', 'W', 'PH', 'PHB', 'T', 'THM', 'HFX_FORCE', 'LH_FORCE', 'TSK_FORCE', 'HFX_FORCE_TEND', 'LH_FORCE_TEND', 'TSK_FORCE_TEND', 'MU', 'MUB', 'NEST_POS', 'P', 'PB', 'FNM', 'FNP', 'RDNW', 'RDN', 'DNW', 'DN', 'CFN', 'CFN1', 'THIS_IS_AN_IDEAL_RUN', 'P_HYD', 'Q2', 'T2', 'TH2', 'PSFC', 'U10', 'V10', 'RDX', 'RDY', 'RESM', 'ZETATOP', 'CF1', 'CF2', 'CF3', 'ITIMESTEP', 'XTIME', 'QVAPOR', 'QCLOUD', 'QRAIN', 'QICE', 'QSNOW', 'QGRAUP', 'RE_CLOUD_GSFC', 'RE_RAIN_GSFC', 'RE_ICE_GSFC', 'RE_SNOW_GSFC', 'RE_GRAUPEL_GSFC', 'RE_HAIL_GSFC', 'QNICE', 'QNSNOW', 'QNRAIN', 'QNGRAUPEL', 'SHDMAX', 'SHDMIN', 'SNOALB', 'TSLB', 'SMOIS', 'SH2O', 'SMCREL', 'SEAICE', 'XICEM', 'SFROFF', 'UDROFF', 'IVGTYP', 'ISLTYP', 'VEGFRA', 'GRDFLX', 'ACGRDFLX', 'ACSNOM', 'SNOW', 'SNOWH', 'CANWAT', 'SSTSK', 'COSZEN', 'LAI', 'VAR', 'MAPFAC_M', 'MAPFAC_U', 'MAPFAC_V', 'MAPFAC_MX', 'MAPFAC_MY', 'MAPFAC_UX', 'MAPFAC_UY', 'MAPFAC_VX', 'MF_VX_INV', 'M

In [10]:
# informacoes das variaveis U10 e V10 (para calculo da direcao do vento)
print(dataset.variables['U10'])

<class 'netCDF4._netCDF4.Variable'>
float32 U10(Time, south_north, west_east)
    FieldType: 104
    MemoryOrder: XY 
    description: U at 10 M
    units: m s-1
    stagger: 
    coordinates: XLONG XLAT XTIME
unlimited dimensions: Time
current shape = (34, 139, 129)
filling on, default _FillValue of 9.969209968386869e+36 used


In [7]:
print(dataset.variables['V10'])

<class 'netCDF4._netCDF4.Variable'>
float32 V10(Time, south_north, west_east)
    FieldType: 104
    MemoryOrder: XY 
    description: V at 10 M
    units: m s-1
    stagger: 
    coordinates: XLONG XLAT XTIME
unlimited dimensions: Time
current shape = (34, 139, 129)
filling on, default _FillValue of 9.969209968386869e+36 used


In [17]:
# obtem os pontos das componentes U e V, e as coordenadas LAT e LONG da grade de pontos
# a dimensao padrao dos pontos eh 34 x 139 x 129 (tempo, lat, long);
# como sao muitos pontos para a area, reduziremos para 1/3 do total de pontos.
# obs: - tempo = 0
#      - 126 eh o maior valor divisivel por 3 de 129
#      - dimensao final: 46 x 42

# max_lat_index = 138
# max_lng_index = 126
# mdc = 6

# max_lat_index = 136
# max_lng_index = 128
# mdc = 8

# max_lat_index = 120
# max_lng_index = 120
# mdc = 40

# var_aux = np.take(dataset.variables['U10'][0], [i for i in range(0, max_lat_index+1, mdc)], axis=0)
# var_u = np.take(var_aux, [i for i in range(0, max_lng_index+1, mdc)], axis=1)

# var_aux = np.take(dataset.variables['V10'][0], [i for i in range(0, max_lat_index+1, mdc)], axis=0)
# var_v = np.take(var_aux, [i for i in range(0, max_lng_index+1, mdc)], axis=1)

# var_aux = np.take(dataset.variables['XLAT'][0], [i for i in range(0, max_lat_index+1, mdc)], axis=0)
# var_lats = np.take(var_aux, [i for i in range(0, max_lng_index+1, mdc)], axis=1)

# var_aux = np.take(dataset.variables['XLONG'][0], [i for i in range(0, max_lat_index+1, mdc)], axis=0)
# var_longs = np.take(var_aux, [i for i in range(0, max_lng_index+1, mdc)], axis=1)

In [41]:
var_u = dataset.variables['U10'][0]
var_v = dataset.variables['V10'][0]
var_lats = dataset.variables['XLAT'][0]
var_longs = dataset.variables['XLONG'][0]

In [19]:
var_u.size

16

In [20]:
# Obtem os valores min e max da latitude
max_lat = var_lats[0][0]
min_lat = var_lats[len(var_lats[0])-1][0]
print('LAT min/max: {} / {}'.format(min_lat, max_lat))

LAT min/max: -17.3173828125 / -34.680599212646484


In [21]:
# Obtem os valores min e max da longitude
max_long = var_longs[0][0]
min_long = var_longs[0][len(var_longs[0])-1]
print('LONG min/max: {} / {}'.format(min_long, max_long))

LONG min/max: -40.103416442871094 / -59.53180694580078


In [22]:
# os pontos superior-esquerdo (TL) e inferior-direito (BR) da grade de pontos
tl_lat = float(min_lat)
tl_long = float(max_long)
print('Top-Left: {} , {}'.format(tl_lat, tl_long))

Top-Left: -17.3173828125 , -59.53180694580078


In [23]:
br_lat = float(max_lat)
br_long = float(min_long)
print('Bottom-Right: {} , {}'.format(br_lat, br_long))

Bottom-Right: -34.680599212646484 , -40.103416442871094


In [24]:
# variaveis de cabecalho para VENTO
nx = var_u.shape[0]
ny = var_u.shape[1]
dx = abs(max_lat - min_lat) / nx
dy = abs(max_long - min_long) / ny
tot = nx * ny
dx

4.340804100036621

In [25]:
nx

4

In [26]:
# dados do componente U do vento
a = var_u
A = np.matrix(a)
b = A.flatten()
c = np.ravel(b).T
u_data = c.tolist()

In [27]:
# dados do componente V do vento
a = var_v
A = np.matrix(a)
b = A.flatten()
c = np.ravel(b).T
v_data = c.tolist()

In [28]:
# obtem a data/hora de referencia do dataset
data_hora = dataset.START_DATE.replace('_', ' ')
data_hora

'2020-10-13 12:00:00'

In [29]:
# formato JSON para VENTO (padrao leaflet-velocity)
wind_data = [{
  "header": {
    "parameterNumberName": "eastward_wind",
    "parameterUnit": "m.s-1",
    "parameterNumber": 2,
    "parameterCategory": 2,
    "nx": nx,
    "ny": ny,
    "numberPoints": tot,
    "dx": dx,
    "dy": dy,
    "la1": tl_lat,
    "lo1": tl_long,
    "la2": br_lat,
    "lo2": br_long,
    "refTime": data_hora
  },
  "data": u_data
}, {
  "header": {
    "parameterNumberName": "northward_wind",
    "parameterUnit": "m.s-1",
    "parameterNumber": 3,
    "parameterCategory": 2,
    "nx": nx,
    "ny": ny,
    "numberPoints": tot,
    "dx": dx,
    "dy": dy,
    "la1": tl_lat,
    "lo1": tl_long,
    "la2": br_lat,
    "lo2": br_long,
    "refTime": data_hora
  },
  "data": v_data
}]

In [30]:
# exporta para o arquivo JSON no padrao esperado pelo Leaflet (leaflet-velocity)
with open('wind.json', 'w') as outfile:  
    json.dump(wind_data, outfile, separators=(',', ':'))

In [92]:
print(dataset.variables['XLAT'])

<class 'netCDF4._netCDF4.Variable'>
float32 XLAT(Time, south_north, west_east)
    FieldType: 104
    MemoryOrder: XY 
    description: LATITUDE, SOUTH IS NEGATIVE
    units: degree_north
    stagger: 
    coordinates: XLONG XLAT
unlimited dimensions: Time
current shape = (34, 139, 129)
filling on, default _FillValue of 9.969209968386869e+36 used


In [5]:
### loop through and create array
# temp is scaled from Kelvin to 0-1 with range 200K to 350K
# USER can edit display options
var_lats = dataset.variables['XLAT'][0]
var_longs = dataset.variables['XLONG'][0]

temps = dataset.variables['T'][0][0]
lats = var_lats
lons = var_longs
        
temp_data = [[0,0,0] for i in range(len(lons) * len(lats))]
for i in range(len(lons)):
    for j in range(len(lats)):
        temp_data[j + (i * len(lats))][0] = lons[i]
        temp_data[j + (i * len(lats))][1] = lats[j]
        temp_data[j + (i * len(lats))][2] = (temps[i][j] - 273.15) # + 273.15 for K
        #temp_data[j + (i * len(lons))][2] = str((temps[i,j] - 200)/150) if string is necessary

IndexError: index 129 is out of bounds for axis 0 with size 129

In [84]:
len(lats)


139

In [78]:
# apply non-overlapping moving window average to reduce data size by factor of 144 
# USER can edit grouping parameter
# number of points should not be more than several hundred for best performance
group = 12
lats_sm = lats.reshape(-1, group).mean(axis=1)
lons_sm = lons.reshape(-1, group).mean(axis=1)

ValueError: cannot reshape array of size 17931 into shape (12)

In [32]:
# create new smaller temperature array 
temp_array = [[0] for i in range(len(lats) * len(lons))]
for i in range(len(temp_data)):
    temp_array[i] = temp_data[i][2]
temps_sm = np.array(temp_array).reshape(-1, group * group).mean(axis=1)

<ipython-input-32-fad9abf35c3b>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  temps_sm = np.array(temp_array).reshape(-1, group * group).mean(axis=1)


ValueError: cannot reshape array of size 19321 into shape (144)

In [33]:
# reformat array to [lat, lon, temp]
temp_data_sm = [[0,0,0] for i in range(len(lats_sm) * len(lons_sm))]
for i in range(len(lats_sm)):
    for j in range(len(lons_sm)):
        temp_data_sm[j + (i * len(lons_sm))][0] = lats_sm[i]
        temp_data_sm[j + (i * len(lons_sm))][1] = lons_sm[j] -180
        temp_data_sm[j + (i * len(lons_sm))][2] = temps_sm[j + (i * len(lons_sm))] + 40

NameError: name 'lats_sm' is not defined

In [ ]:
# write Javascript file for Leaflet.idw input
with open('temps_sm.js', 'w') as filehandle:  
    filehandle.write('var addressPoints = ' + str(temp_data_sm))